In [1]:
from run import *

In [2]:
def load_content(args):
    df = pd.read_csv(os.path.join(args.root_path, 'prompt_bank.csv'))
    data_name = args.data_path.split('.')[0] 
    content = df[df['data']==data_name]['prompt'].values[0]
    return content
            
def get_parser():

    parser = get_basic_parser("TimeLLM")

    parser.add_argument(
        '--model_id', default='ori', choices=['ori', 'removeLLM', 
        'randomInit', 'llm_to_trsf', 'llm_to_attn']
    )
    parser.add_argument('--model', type=str, default='TimeLLM', choices=['TimeLLM'])
    
    parser.add_argument('--patch_len', type=int, default=16, help='patch length')
    parser.add_argument('--stride', type=int, default=8, help='stride')
    parser.add_argument('--prompt_domain', type=int, default=1, help='')
    parser.add_argument(
        '--llm_model', type=str, default='GPT2', help='LLM model',
        choices=['LLAMA', 'GPT2', 'BERT']) # 
    parser.add_argument('--llm_dim', type=int, default='768', 
        help='LLM model dimension. LLama7b:4096; GPT2-small:768; BERT-base:768')
    parser.add_argument('--llm_layers', type=int, default=6)
    
    return parser

In [12]:
argv = """
--n_features 1 --d_model 16 --features S 
--data_path Financial_Aid_State.csv --group_id GROUP_ID --freq a 
--batch_size 16 --itrs 3 --disable_progress
--seq_len 10 --label_len 5 --pred_len 1
--model_id ori --top_k 2
--patch_len 4 --stride 2 --target need_amt --dump_output --itr_no 1
""".split()

parser = get_parser()
args = parser.parse_args(argv)
args.content = load_content(args)

In [13]:
initial_setup(args)
set_random_seed(args.seed)

NVIDIA GeForce GTX 1650


In [14]:
parent_seed = args.seed
np.random.seed(parent_seed)
experiment_seeds = np.random.randint(1e3, size=args.itrs)
experiment_seeds = [int(seed) for seed in experiment_seeds]
args.experiment_seeds = experiment_seeds
original_itr = args.itr_no

In [15]:
itr_no = args.itr_no
            
args.seed = experiment_seeds[itr_no-1]
print(f'\n>>>> itr_no: {itr_no}, seed: {args.seed} <<<<<<')
set_random_seed(args.seed)
args.itr_no = itr_no

exp = Exp_Long_Term_Forecast(args)

print('\n>>>>>>> testing :  <<<<<<<<<<<<<<<<<<<')
exp.test(flag='test', dump_output=args.dump_output)


>>>> itr_no: 1, seed: 648 <<<<<<
Use GPU: cuda:0
description: Yearly funding Aid to each USA state to support undergraduate education.
Experiments will be saved in results\Financial_Aid_State\TimeLLM_sl_10_pl_1_id_ori\1

Experiment begins at 2024-09-26 11:43:36


>>>>>>> testing :  <<<<<<<<<<<<<<<<<<<
Scaling data.
test 92
Loading model from results\Financial_Aid_State\TimeLLM_sl_10_pl_1_id_ori\1\llm.pth


RuntimeError: Error(s) in loading state_dict for Model:
	size mismatch for output_projection.linear.weight: copying a param with shape torch.Size([1, 384]) from checkpoint, the shape in current model is torch.Size([1, 640]).

In [16]:
4 * 128

512

In [ ]:
test_data, test_loader = exp.get_data('test')
exp.load_best_model()
model = exp.model
device = exp.device

disable_progress = args.disable_progress

In [ ]:
preds = []
trues = []
f_dim = -1 if args.features == 'MS' else 0

model.eval()
with torch.no_grad():
    for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in tqdm(
        enumerate(test_loader), desc="Running inference",
        total=len(test_loader)
    ):
        batch_x = batch_x.float().to(device)
        batch_y = batch_y.float().to(device)

        batch_x_mark = batch_x_mark.float().to(device)
        batch_y_mark = batch_y_mark.float().to(device)

        # decoder input
        dec_inp = torch.zeros_like(batch_y[:, -args.pred_len:, :]).float()
        dec_inp = torch.cat([batch_y[:, :args.label_len, :], dec_inp], dim=1).float().to(device)

        if args.model == 'CALF':
            outputs = model(batch_x)
            outputs = outputs['outputs_time']
        elif args.model == 'OFA':
            outputs = model(batch_x)
        else:
            outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)

            if args.output_attention: outputs = outputs[0]

        outputs = outputs[:, -args.pred_len:, f_dim:].detach().cpu().numpy()
        batch_y = batch_y[:, -args.pred_len:, f_dim:].detach().cpu().numpy()

        preds.append(outputs)
        trues.append(batch_y)

# this line handles different size of batch. E.g. last batch can be < batch_size.
preds = np.concatenate(preds, axis=0)
trues = np.concatenate(trues, axis=0)

In [ ]:
for i in range(preds.shape[0]):
    # date = test_data.index.loc[i, 'date']
    scaler = test_data.scaler[i]
    preds[i] = test_data.inverse_transform(scaler, preds[i])
    trues[i] = test_data.inverse_transform(scaler, trues[i])

In [ ]:
print("Removing negatives...")
preds[preds<0] = 0